<a href="https://colab.research.google.com/github/rashmi2825/FitPluse-Health-Anomaly-Detection-from-Fitness-Devices/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Loading and Verification

In this step, we connect Google Drive with Google Colab and verify the availability of the required Fitbit datasets.
The datasets include daily activity data, heart rate data, and sleep data, which will be used for health anomaly detection analysis.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

base_path = "/content/drive/MyDrive/FitPulse_Milestone1"
os.listdir(base_path)


In [ ]:
!pip install pandas numpy   --quiet

In [ ]:
import pandas as pd


In [ ]:
pd.__version__


In [ ]:
import pandas as pd
activity = pd.read_csv("/content/drive/MyDrive/FitPulse_Milestone1/dailyActivity_merged.csv")
activity.head()


In [ ]:
activity.shape


In [ ]:
activity.columns

In [ ]:
activity['ActivityDate'] = pd.to_datetime(activity['ActivityDate'])

In [ ]:
activity_clean = activity[['Id', 'ActivityDate', 'TotalSteps']]
activity_clean.head()


In [ ]:
activity_clean.isnull().sum()

In [ ]:
activity_clean['TotalSteps'].describe()


In [ ]:
activity_clean = activity_clean.sort_values('ActivityDate')
activity_clean.head()


In [ ]:
activity_clean = activity_clean.set_index('ActivityDate')
activity_clean.head()


In [ ]:
activity_clean = activity_clean.sort_index()
activity_clean.head()


In [ ]:
activity_clean.index.is_monotonic_increasing

In [ ]:
heart = pd.read_csv("/content/drive/MyDrive/FitPulse_Milestone1/heartrate_seconds_merged.csv")
heart.head()

In [ ]:
heart.columns

In [ ]:
heart['Time'] = pd.to_datetime(heart['Time'])

In [ ]:
heart.dtypes

In [ ]:
heart = heart.sort_values('Time')
heart.head()

In [ ]:
heart = heart.set_index('Time')

In [ ]:
heart.head()

In [ ]:
heart.index.is_monotonic_increasing

In [ ]:
sleep = pd.read_csv("/content/drive/MyDrive/FitPulse_Milestone1/minuteSleep_merged.csv")


In [ ]:
sleep.head()

In [ ]:
sleep.columns

In [ ]:
sleep['date'] = pd.to_datetime(sleep['date'])

In [ ]:
sleep.dtypes

In [ ]:
sleep = sleep.sort_values('date')
sleep.head()

In [ ]:
sleep = sleep.set_index('date')
sleep.index.is_monotonic_increasing

In [ ]:
heart_daily = heart.groupby(['Id', heart.index.date])['Value'].mean().reset_index()
heart_daily.columns = ['Id', 'Date', 'AvgHeartRate']

In [ ]:
heart_daily.head()

In [ ]:
steps_daily = activity_clean.reset_index()
steps_daily.rename(columns={'ActivityDate': 'Date'}, inplace=True)

In [ ]:
steps_daily.head()

In [ ]:
sleep_daily = sleep.groupby(['Id', sleep.index.date])['value'].sum().reset_index()
sleep_daily.columns = ['Id', 'Date', 'TotalSleepMinutes']

In [ ]:
sleep_daily.head()

In [ ]:
steps_daily['Date'] = pd.to_datetime(steps_daily['Date'])
heart_daily['Date'] = pd.to_datetime(heart_daily['Date'])
sleep_daily['Date'] = pd.to_datetime(sleep_daily['Date'])

In [ ]:
merged_1 = pd.merge(
    steps_daily,
    heart_daily,
    on=['Id', 'Date'],
    how='inner'
)

In [ ]:
merged_1.head()
merged_1.shape

In [ ]:
final_data = pd.merge(
    merged_1,
    sleep_daily,
    on=['Id', 'Date'],
    how='inner'
)

In [ ]:
final_data.head()
final_data.shape

In [ ]:
final_data.to_csv(
    "/content/drive/MyDrive/FitPulse_Milestone1/final_merged_data.csv",
    index=False
)

In [44]:
### Minute-Level Data Preprocessing


In [48]:
# --- Minute-level Heart Rate ---
heart = pd.read_csv("/content/drive/MyDrive/FitPulse_Milestone1/heartrate_seconds_merged.csv")
heart['timestamp'] = pd.to_datetime(heart['Time'])
heart = heart.rename(columns={'Value': 'HeartRate'})
heart = heart[['Id', 'timestamp', 'HeartRate']]

# --- Daily Steps (mapped to minutes) ---
steps_daily = activity[['Id', 'ActivityDate', 'TotalSteps']].copy()
steps_daily['Date'] = pd.to_datetime(steps_daily['ActivityDate'])
steps_daily = steps_daily[['Id', 'Date', 'TotalSteps']]

heart['Date'] = heart['timestamp'].dt.date
heart['Date'] = pd.to_datetime(heart['Date'])

heart_steps = pd.merge(
    heart,
    steps_daily,
    on=['Id', 'Date'],
    how='left'
)

# --- Minute-level Sleep ---
sleep = pd.read_csv("/content/drive/MyDrive/FitPulse_Milestone1/minuteSleep_merged.csv")
sleep['timestamp'] = pd.to_datetime(sleep['date'])
sleep = sleep.rename(columns={'value': 'SleepFlag'})
sleep = sleep[['Id', 'timestamp', 'SleepFlag']]

# --- Final Minute Merge ---
final_minute_data = pd.merge(
    heart_steps,
    sleep,
    on=['Id', 'timestamp'],
    how='left'
)

final_minute_data['SleepFlag'] = final_minute_data['SleepFlag'].fillna(0)


In [49]:
final_minute_data.to_csv(
    "/content/drive/MyDrive/FitPulse_Milestone1/data/minute_level_data.csv",
    index=False
)
